In [1]:
# https://drive.google.com/file/d/1c0irqiDFwTvhEq845ZLQCqo6cvW_JNfL/view?usp=sharing
import gdown
file_id = '1c0irqiDFwTvhEq845ZLQCqo6cvW_JNfL'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'data.zip', quiet=False)
!unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1c0irqiDFwTvhEq845ZLQCqo6cvW_JNfL
To: /content/data.zip
100%|██████████| 13.9M/13.9M [00:00<00:00, 54.7MB/s]


Archive:  data.zip
   creating: sales_predit/
  inflating: sales_predit/items.csv  
  inflating: sales_predit/item_categories.csv  
  inflating: sales_predit/sales_train.csv  
  inflating: sales_predit/shops.csv  


- 시차 피처 : time lag feature
  - 과거시점에 대한 피처
  - 현시점 데이터에 과저 시점 데이터를 추가
  - 과거시점 데이터는 향후 판매량 예측에 유용
- 생성절차
  - 기준 피처별 시차피처 생성
  - 기준은 다양하게 정의
  - 함수
    - 상품id별 평균 판매량
    - (상품id + 도시  ) 평균 판매량
    - (상점id + 상품범주id) 평균 판매
- 기준피처는 월ID
  - 너무세분화하면 X -> 2개나 3개로 정의    

In [2]:
import pandas as pd
sales_train = pd.read_csv('/content/sales_predit/sales_train.csv')
item_categories = pd.read_csv('/content/sales_predit/item_categories.csv')
items = pd.read_csv('/content/sales_predit/items.csv')
shops = pd.read_csv('/content/sales_predit/shops.csv')

sales_train = sales_train.rename(columns={'date': '날짜',
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

In [3]:
# 메모리 사용량이 많은거 같아서 다운캐스팅
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))

    return df

all_df = [sales_train, shops, items, item_categories]
for df in all_df:
    df = downcast(df)

54.2% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨


In [4]:
# 이상치 제거
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가'] > 0]
# 판매가가 50,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가'] < 50000]

# 판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매량'] > 0]
# 판매량이 1,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매량'] < 1000]

In [5]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [6]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()
# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [7]:
# 상점명 피처 제거
shops = shops.drop('상점명', axis=1)

shops.head()

,상점ID,도시
0,0,0
1,1,0
2,2,1
3,3,2
4,4,3


In [8]:
# 상품명 피처 제거
items = items.drop(['상품명'], axis=1)

In [9]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID': 'min'})['월ID']

items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [10]:
sales_train['월ID'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int8)

In [11]:
# 첫 판매월 피처의 결측값을 33로 대체
items['첫 판매월'] = items['첫 판매월'].fillna(33)

In [12]:
# 상품분류명의 첫 단어를 대분류로 추출
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

In [13]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# 대분류의 고윳값 개수가 5개 미만이면 'etc'로 바꾸기
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [14]:
# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 대분류 피처 레이블 인코딩
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

# 상품분류명 피처 제거
item_categories = item_categories.drop('상품분류명', axis=1)

In [15]:
from itertools import product
import numpy as np
train = []
# 월ID, 상점ID, 상품ID 조합 생성
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [16]:
# 파생피처 생
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '월간 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

In [17]:
import gc

# group 변수 가비지 컬렉션
del group
gc.collect();

In [18]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

# 가비지 컬렉션
del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


In [19]:
all_data = train.copy()
# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [20]:
# 모든 데이터 병합
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

65.5% 압축됨


In [21]:
# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

In [22]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 피처 확인
    assert (idx_features[0] == '월ID') and \
           len(idx_features) in [2, 3]

    # 파생 피처명 설정
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'

    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기
    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량': feature_name})

    # df와 group 병합
    df = df.merge(group, on=idx_features, how='left')
    # 데이터 다운캐스팅
    df = downcast(df, verbose=False)
    # 새로 만든 feature_name 피처명을 mean_features 리스트에 추가
    mean_features.append(feature_name)

    # 가비지 컬렉션
    del group
    gc.collect()

    return df, mean_features

In [23]:
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
item_mean_features = []

# ['월ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])

# ['월ID', '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [24]:
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 피처명을 담을 리스트
shop_mean_features = []

# ['월ID', '상점ID', '상품분류ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(df=all_data,
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [25]:
def add_lag_features(df, lag_features_to_clip, idx_features,
                     lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사
    df_temp = df[idx_features + [lag_feature]].copy()

    # 시차 피처 생성
    for i in range(1, nlags+1):
        # 시차 피처명
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp의 date_block_num 피처에 1 더하기
        df_temp['월ID'] += 1
        # idx_feature를 기준으로 df와 df_temp 병합하기
        df = df.merge(df_temp.drop_duplicates(),
                      on=idx_features,
                      how='left')
        # 결측값 0으로 대체
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # 0 ~ 20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가
        if clip:
            lag_features_to_clip.append(lag_feature_name)

    # 데이터 다운캐스팅
    df = downcast(df, False)
    # 가비지 컬렉션
    del df_temp
    gc.collect()

    return df, lag_features_to_clip

In [26]:
lag_features_to_clip = [] # 0 ~ 20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 월간 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량',
                                                  nlags=3,
                                                  clip=True) # 값을 0 ~ 20 사이로 제한

In [27]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수',
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가',
                                                  nlags=3)

In [28]:
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip,
                                                      idx_features=idx_features,
                                                      lag_feature=item_mean_feature,
                                                      nlags=3,
                                                      clip=True)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

In [29]:
# ['월ID', '상점ID', '상품분류ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip,
                                                      idx_features=['월ID', '상점ID', '상품분류ID'],
                                                      lag_feature=shop_mean_feature,
                                                      nlags=3,
                                                      clip=True)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

In [30]:
# 월ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

In [31]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2',
                                          '월간 판매량_시차3']].mean(axis=1)

In [32]:
# 0 ~ 20 사이로 값 제한
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [33]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf],
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf],
                                                        np.nan).fillna(0)

In [34]:
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

In [35]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [36]:
all_data['월'] = all_data['월ID'] % 12

In [37]:
# 첫 판매월, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

In [38]:
all_data = downcast(all_data, False) # 데이터 다운캐스팅

In [39]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 32]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 32]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# 테스트 데이터(피처)
X_test = all_data[all_data['월ID'] == 33]
X_test = X_test.drop(['월간 판매량'], axis=1)
y_test = all_data[all_data['월ID'] == 33]['월간 판매량']


# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 32]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 32]['월간 판매량']

# 가비지 컬렉션
del all_data
gc.collect();

In [40]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터
import lightgbm as lgb
# 인기있는 파라메터
params = {
    'objective': 'regression',  # or 'binary' for classification
    'metric': 'rmse',          # or 'auc' for classification
    'boosting_type': 'gbdt',    # or 'dart' or 'goss'
    'num_leaves': 31,          # number of leaves in each tree
    'learning_rate': 0.05,     # learning rate
    'feature_fraction': 0.9,   # fraction of features to use in each tree
    'bagging_fraction': 0.8,   # fraction of data to use in each tree
    'bagging_freq': 5,         # frequency of bagging
    'verbose': 0,              # verbosity level
    'seed': 42                # random seed for reproducibility
}

cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


lgb_model = lgb.train(params, dtrain,
                      categorical_feature=cat_features,
                      valid_sets=(dtrain,dvalid),
                      num_boost_round=500,
                      callbacks=[lgb.early_stopping(100)] ,
                      )


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[497]	training's rmse: 0.788893	valid_1's rmse: 0.85581


In [41]:
from sklearn.metrics import r2_score
y_pred = lgb_model.predict(X_test)
r2_score(y_test, y_pred)

NameError: name 'all_data' is not defined